# Import the dependencies

In [1]:
import os
from mieze_python.mieze import Mieze
from pprint import pprint
import sys 
print(sys.version)

'''
###########################################################
The manager is the instance linking the following parts:
- I/O manager for input output
- The processing of data 
- The fitting of data (maybe merge processing and fitting)
- The graphical manopulation of reduced and fitted data
###########################################################
'''
tool = Mieze()



3.7.0 (default, Aug 20 2018, 21:19:42) 
[Clang 9.1.0 (clang-902.0.39.2)]


# Process the elastic data

## First set up the environnement

In [2]:
'''
###########################################################
INITIALISE THE ENVIRONEMENT:

Each dataset is initialised in a new environement. We
therefore initialise a new environement through the manager
run command with the keyword 'new environement'. The
title and selected type are optional and will fall back to 
'No_title' and 'MIEZE' if not provided.

###########################################################
'''
elastic_env = tool.new_environment(title = 'elastic', select = 'SANS')


## Load the data

In [3]:
'''
###########################################################
LOAD THE DATA:

at a further point in time different routines will be 
provided to load data. The current way of proceedign is to 
load a file containing all relevant information and let the
routine parse the content. 
###########################################################
'''
elastic_env.io.load_SANS_PAD('SkX_down.txt')
elastic_env.process.extract_from_metadata(axis = 'Temperature', key = 'T_B')
print(elastic_env.current_data)

=========== This is the Input ============

['Data', "['110_1']", '0012', 'range(2194, 2298)', '(128x128)']
['Data', "['100_1']", '0012', 'range(2681, 2725)', '(128x128)']
['Data', "['100_2']", '0012', 'range(2890, 2956)', '(128x128)']
['Data', "['BG']", '0012', 'range(1609, 1611)', '(128x128)']

##########################################################
################## DATA STRUCTURE ########################
##########################################################
The datastructure currently consists of:
- Number of elements: 108
- Number of dimensions: 2
- The axes names are: ['Type', 'Temperature']
- The axes units are: ['-', 'K']
- The axes lengths are: [4, 106]
- The base structure is: np
- The base structure dim is: (128, 128)
- The metadata addition is set to: True
----------------------------------------------------------
Creation date: Tue Nov 20 08:05:39 2018
Source format: PAD files
Measurement type: SANS
##########################################################




## Create a Mask anprocess intensities

In [4]:
'''
###########################################################
SET THE MASK:

A large variety of masks are available to be edited. Using
the 'set mask template' keyword it is possible to set a 
predefined mask. Further implementations will allow the 
change of the default mask parameters and at some point the 
loading of custome masks from file.
###########################################################
'''
elastic_env.mask.select_template(key = 'SkX_peak_SkXCon')
elastic_env.process.calculate_intensity()
elastic_result_1 = elastic_env.get_result(name = 'Intensity')

elastic_env.mask.select_template(key = 'SkX_peak_Sixfold')
elastic_env.process.calculate_intensity()
elastic_result_2 = elastic_env.get_result(name = 'Intensity')


/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_sans.py:152: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_sans.py:155: RuntimeWarning: invalid value encountered in double_scalars


# Vertical data

## Create environement and load data

In [5]:
vertical_env = tool.new_environment(title = 'vertical MIEZE', select = 'MIEZE')
vertical_env.io.load_MIEZE_TOF('LoadSkXCon.txt')
print(vertical_env.current_data)

=========== This is the Input ============

['Data', '[28.40,0]', '0012', 'range(2727, 2736)', '(8x16x128x128)']
['Data', '[62.00,0]', '0012', '[2736]+[i for i in range(2739, 2747)]', '(8x16x128x128)']
['Data', '[28.60,0]', '0012', 'range(2747, 2756)', '(8x16x128x128)']
['Data', '[28.80,0]', '0012', 'range(2813, 2822)', '(8x16x128x128)']
['Data', '[28.95,0]', '0012', 'range(2774, 2783)', '(8x16x128x128)']
['Data', '[29.05,0]', '0012', 'range(2804, 2813)', '(8x16x128x128)']
['Data', '[29.15,0]', '0012', 'range(2765, 2774)', '(8x16x128x128)']
['Data', '[29.25,0]', '0012', 'range(2858, 2867)', '(8x16x128x128)']
['Data', '[29.35,0]', '0012', 'range(2756, 2765)', '(8x16x128x128)']
['Data', '[29.45,0]', '0012', 'range(2849, 2858)', '(8x16x128x128)']
['Data', '[29.60,0]', '0012', 'range(2792, 2801)', '(8x16x128x128)']
['Data', '[29.75,0]', '0012', 'range(2783, 2792)', '(8x16x128x128)']
['Data', '[30.50,0]', '0012', 'range(2840, 2849)', '(8x16x128x128)']

######################################

## Set the mask

In [6]:
vertical_env.mask.select_template(key = 'Pre_SkX_peak_SkXCon')
print(vertical_env.mask)


##########################################################
################## MASK STRUCTURE ########################
##########################################################
The mask has been set as follows: 
- Selected mask template: Pre_SkX_peak_SkXCon
- Parameters:
          (28, 34)
          65
          85
          (80, 104)
          10
          11
----------------------------------------------------------
##########################################################




## Set the fitting parameters

In [7]:
'''
###########################################################
SET THE FITING PARAMETERS:

Each fit class has anumber of fit parameters that can be 
set. This is doen in the following. 
###########################################################
'''
foils_in_echo = []
for i in range(5):
    foils_in_echo.append([1,1,1,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([0,0,0,0,1,0])
foils_in_echo.append([0,0,0,0,1,0])

#set the values to be processed as data
Select = [28.40, 28.60, 28.80, 28.95, 29.05, 29.15, 29.25, 29.35, 29.45, 29.60, 29.75]

#set the reference value
Reference = [28.6,0]

#set the background
Background = None

vertical_env.fit.set_parameter( name = 'Select',        value = Select       )
vertical_env.fit.set_parameter( name = 'Reference',     value = Reference    )
vertical_env.fit.set_parameter( name = 'Background',    value = Background   )
vertical_env.fit.set_parameter( name = 'foils_in_echo', value = foils_in_echo)


## Process throught the fitting steps

In [8]:
'''
###########################################################
PROCESS THE DATA:


The process class maanges the workflows as no fit class
funciton should be called dierectly. All results will be 
put into the result class and logs into the log class
linked to the result. 
###########################################################
'''
vertical_env.process.calculate_echo()
vertical_env.process.remove_foils()
vertical_env.process.calculate_shift()
vertical_env.mask.select_template(key = 'SkX_peak_SkXCon')

######COUNTS##### 1918
######COUNTS##### 713
######COUNTS##### 1165
######COUNTS##### 2330
######COUNTS##### 2020
######COUNTS##### 3710
######COUNTS##### 1951
######COUNTS##### 738
######COUNTS##### 1171
######COUNTS##### 2293
######COUNTS##### 2024
######COUNTS##### 3794
######COUNTS##### 2009
######COUNTS##### 708
######COUNTS##### 1181
######COUNTS##### 2294
######COUNTS##### 2067
######COUNTS##### 3856
######COUNTS##### 2035
######COUNTS##### 787
######COUNTS##### 1177
######COUNTS##### 2361
######COUNTS##### 2044
######COUNTS##### 3741
######COUNTS##### 1980
######COUNTS##### 774
######COUNTS##### 1132
######COUNTS##### 2377
######COUNTS##### 2095
######COUNTS##### 3839
######COUNTS##### 2011
######COUNTS##### 766
######COUNTS##### 1194
######COUNTS##### 2373
######COUNTS##### 2067
######COUNTS##### 3704
######COUNTS##### 3963
######COUNTS##### 1542
######COUNTS##### 2382
######COUNTS##### 4572
######COUNTS##### 4115
######COUNTS##### 7627
######COUNTS##### 4179
######COUNTS#####

In [9]:
#vertical_env.mask.set_parameters([[28,34], 65, 85,[80, 104]])
print(vertical_env.mask)
#tool.launch_sp(vertical_env)




##########################################################
################## MASK STRUCTURE ########################
##########################################################
The mask has been set as follows: 
- Selected mask template: SkX_peak_SkXCon
- Parameters:
          (28, 34)
          65
          20
          (82, 100)
----------------------------------------------------------
##########################################################




In [10]:
vertical_env.process.calculate_ref_contrast()
vertical_env.process.calculate_contrast()
vert_result = vertical_env.get_result(name = 'Contrast fit')


Processing the reference contrast calculation for: [28.6, 0]K
Processing the contrast fit for: 28.6K measurement 0.0-
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####COMBINE 0.0
2018-11-20 08:05:57.260493 WARNING Fit failed: No counts present.
2018-11-20 08:05:57.260544 ERROR Fit failed: No counts present.
2018-11-20 08:05:57.260612 ERROR Could not find the key: ampl, returning 0
2018-11-20 08:05:57.260636 ERROR Could not find the key: ampl_error, returning 0
2018-11-20 08:05:57.260656 ERROR Could not find the key: mean, returning 0
2018-11-20 08:05:57.260675 ERROR Could not find the key: mean_error, returning 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####COMBINE 0.0
2018-11-20 08:05:57.267656 WARNING Fit failed: No counts present.
2018-11-20 08:05:57.267683 ERROR Fit failed: No counts present.
2018-11-20 08:05:57.267960 ERROR Could not find the key: ampl, returning 0
2018-11-20 08:05:57.2

#####COMBINE 0.0
2018-11-20 08:05:57.449667 WARNING Fit failed: No counts present.
2018-11-20 08:05:57.449694 ERROR Fit failed: No counts present.
2018-11-20 08:05:57.450585 ERROR Could not find the key: ampl, returning 0
2018-11-20 08:05:57.450610 ERROR Could not find the key: ampl_error, returning 0
2018-11-20 08:05:57.450622 ERROR Could not find the key: mean, returning 0
2018-11-20 08:05:57.450638 ERROR Could not find the key: mean_error, returning 0
Processing the contrast fit for: 28.95K measurement 0.0-
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####COMBINE 0.0
2018-11-20 08:05:57.457513 WARNING Fit failed: No counts present.
2018-11-20 08:05:57.457605 ERROR Fit failed: No counts present.
2018-11-20 08:05:57.457656 ERROR Could not find the key: ampl, returning 0
2018-11-20 08:05:57.457679 ERROR Could not find the key: ampl_error, returning 0
2018-11-20 08:05:57.457699 ERROR Could not find the key: mean, returning 0
2018-11-20 08:05

2018-11-20 08:05:57.651039 ERROR Fit failed: No counts present.
2018-11-20 08:05:57.651152 ERROR Could not find the key: ampl, returning 0
2018-11-20 08:05:57.651177 ERROR Could not find the key: ampl_error, returning 0
2018-11-20 08:05:57.651198 ERROR Could not find the key: mean, returning 0
2018-11-20 08:05:57.651219 ERROR Could not find the key: mean_error, returning 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####COMBINE 0.0
2018-11-20 08:05:57.657716 WARNING Fit failed: No counts present.
2018-11-20 08:05:57.657749 ERROR Fit failed: No counts present.
2018-11-20 08:05:57.657910 ERROR Could not find the key: ampl, returning 0
2018-11-20 08:05:57.657934 ERROR Could not find the key: ampl_error, returning 0
2018-11-20 08:05:57.657955 ERROR Could not find the key: mean, returning 0
2018-11-20 08:05:57.657974 ERROR Could not find the key: mean_error, returning 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0
#####PREMASK 0


####CONTRAST##### [nan nan nan nan nan nan nan nan nan] [0.06236245 0.07160134 0.12671127 0.22533633 0.36585973 0.56634347
 0.83334717 1.57060996 1.88011251] [nan nan nan nan nan nan nan nan nan]
####CONTRAST##### [nan nan nan nan nan nan nan nan nan] [0.06236245 0.07160134 0.12671127 0.22533633 0.36585973 0.56634347
 0.83334717 1.57060996 1.88011251] [nan nan nan nan nan nan nan nan nan]
####CONTRAST##### [nan nan nan nan nan nan nan nan nan] [0.06236245 0.07160134 0.12671127 0.22533633 0.36585973 0.56634347
 0.83334717 1.57060996 1.88011251] [nan nan nan nan nan nan nan nan nan]
####CONTRAST##### [nan nan nan nan nan nan nan nan nan] [0.06236245 0.07160134 0.12671127 0.22533633 0.36585973 0.56634347
 0.83334717 1.57060996 1.88011251] [nan nan nan nan nan nan nan nan nan]
####CONTRAST##### [nan nan nan nan nan nan nan nan nan] [0.06236245 0.07160134 0.12671127 0.22533633 0.36585973 0.56634347
 0.83334717 1.57060996 1.88011251] [nan nan nan nan nan nan nan nan nan]
####CONTRAST##### [n

/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_mieze_ctrst.py:593: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_mieze_ctrst.py:595: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_mieze_ctrst.py:596: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_mieze_ctrst.py:593: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_mieze_ctrst.py:595: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/site-packages/mieze_python-0.0.1-py3.7.egg/mieze_python/core/fit_modules/fit_mie

In [11]:


vert_result.log.dump_to_file('result.txt')
vert_result.log.dump_to_console()

2018-11-20 08:05:57.847254 INFORMATION Added the entry'Date' to the metadata
2018-11-20 08:05:57.847256 INFORMATION Added the entry'Start' to the metadata
2018-11-20 08:05:57.847257 INFORMATION Added the entry'name' to the metadata
2018-11-20 08:05:57.865229 INFORMATION Added the entry'Gamma' to the results
2018-11-20 08:05:57.865235 INFORMATION Added the entry'Gamma_error' to the results
2018-11-20 08:05:57.865236 INFORMATION Added the entry'Curve' to the results
2018-11-20 08:05:57.865238 INFORMATION Added the entry'Parameters' to the results
2018-11-20 08:05:57.865239 INFORMATION Added the entry'Select' to the results
2018-11-20 08:05:57.865241 INFORMATION Added the entry'BG' to the results
2018-11-20 08:05:57.865242 INFORMATION Added the entry'Reference' to the results
2018-11-20 08:05:57.865251 INFORMATION Added the entry'Axis' to the results
2018-11-20 08:05:57.865253 INFORMATION Added the entry'Axis_unit' to the results
2018-11-20 08:05:57.865255 INFORMATION Fitting of the contr

# Parallel Data

In [12]:
parallel_env = tool.new_environment(title = 'parallel MIEZE', select = 'MIEZE')
parallel_env.io.load_MIEZE_TOF('SkXSixfold.txt')
print(parallel_env.current_data)


=========== This is the Input ============

['Data', '[68.00,0]', '0012', 'range(2186, 2194)', '(8x16x128x128)']
['Data', '[31.00,0]', '0012', 'range(2387, 2395)', '(8x16x128x128)']
['Data', '[30.00,0]', '0012', '[2396, 2398, 2401, 2402, 2403]', '(8x16x128x128)']
['Data', '[30.00,1]', '0012', '[2408, 2409, 2410]', '(8x16x128x128)']
['Data', '[29.80,0]', '0012', '[2412, 2413, 2374, 2375, 2376, 2377, 2378, 2379]', '(8x16x128x128)']
['Data', '[29.60,0]', '0012', '[2320, 2321, 2414, 2323, 2415, 2325, 2326, 2327]', '(8x16x128x128)']
['Data', '[29.40,0]', '0012', 'range(2334, 2342)', '(8x16x128x128)']
['Data', '[29.20,0]', '0012', '[2348, 2349, 2350, 2351, 2352, 2353, 2354]', '(8x16x128x128)']
['Data', '[29.20,1]', '0012', '[2416]', '(8x16x128x128)']
['Data', '[29.10,0]', '0012', 'range(2418, 2426)', '(8x16x128x128)']
['Data', '[29.00,0]', '0012', '[2362, 2363, 2364, 2365, 2366, 2426, 2368, 2369]', '(8x16x128x128)']
['Data', '[28.60,0]', '0012', 'range(2434, 2442)', '(8x16x128x128)']
['Data'

In [13]:
parallel_env.mask.select_template(key = 'Pre_SkX_peak_Sixfold')
parallel_env.mask.add_command(command_str = 'mask.real[abs(mask.real) > 8] = 0')
#parallel_env.mask.set_parameters([[28,34], 45, 90,[17, 56],15,13])
print(parallel_env.mask)



##########################################################
################## MASK STRUCTURE ########################
##########################################################
The mask has been set as follows: 
- Selected mask template: Pre_SkX_peak_Sixfold
- Parameters:
          (28, 34)
          45
          90
          (17, 56)
          15
          13
----------------------------------------------------------
##########################################################




In [14]:
parallel_env.process.calculate_echo()
parallel_env.process.remove_foils()
parallel_env.process.calculate_shift()

TypeError: unsupported operand type(s) for *: 'Data_Structure' and 'bool'

## Set fit parameters

In [ ]:
############################################
#initialize them
foils_in_echo = []
for i in range(4):
    foils_in_echo.append([1,1,1,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([1,1,0,1,1,1])
foils_in_echo.append([0,0,0,0,1,0])
foils_in_echo.append([0,0,0,0,1,0])


#set the values to be processed as data
Select = [ 28.6, 29.0, 29.1, 29.2, 29.4, 29.6, 29.8, 30.0]

#set the reference value
Reference = [28.6,0]

#set the background
Background = 68.0

parallel_env.fit.set_parameter( name = 'Select',        value = Select       )
parallel_env.fit.set_parameter( name = 'Reference',     value = Reference    )
parallel_env.fit.set_parameter( name = 'Background',    value = Background   )
parallel_env.fit.set_parameter( name = 'foils_in_echo', value = foils_in_echo)

## Process to fit the data

In [ ]:
parallel_env.mask.select_template(key = 'SkX_peak_Sixfold')
#parallel_env.mask.set_parameters([[28,34], 45, 90,[17, 56]])
print(parallel_env.mask)


parallel_env.process.calculate_ref_contrast()

#coorect values
parallel_env.results.set_result( 
         name = 'Reference contrast calculation', 
         position = ['Contrast_ref',0.36585973199337996], 
         value = 0.73)

parallel_env.results.set_result(
         name = 'Reference contrast calculation', 
         position = ['Contrast_ref_error',0.36585973199337996], 
         value = 0.0035)

parallel_env.process.calculate_contrast()
parallel_result = parallel_env.get_result(name = 'Contrast fit')


# Plot perp data

In [ ]:
############################################
#see what hapens
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.constants as co
from matplotlib.colors import Colormap, LogNorm
from matplotlib.cm import get_cmap
import pandas as pd
%matplotlib inline
fig = plt.figure(figsize=(16,5))
ax = fig.add_subplot(1,2,1)

Ts = []
Gammas = []
Gammaerrs = []

index = 0
def get_color(n, number_of_colors, cmapstr):

    cmap = get_cmap(cmapstr)

    return cmap(float(n)/number_of_colors)

print(vert_result['Select'])
for T in vert_result['Select']:
    print(vert_result['Parameters'][T]['y'])
    x           = vert_result['Parameters'][T]['x']
    y           = vert_result['Parameters'][T]['y']
    y_error     = vert_result['Parameters'][T]['y_error']

    c = get_color(index,11,'tab10')
    ax.errorbar(
        x, 
        y+1.2-index*0.2,
        y_error, 
        fmt='o', 
        color=c)
        #label='$T=%.2f\,K$' %SkXCon[T].T_B)
    x = np.linspace(0.01,3,1000)

    ax.plot(x, vert_result['Curve'][T]+1.2-index*0.2, color=c)
    
    index += 1


ax.set_xscale('log')
ax.set_xlim(0.01,3)
ax.legend(bbox_to_anchor=(-0.1,0.9))
ax.text(2e-2, 0, r'$B \perp n$, $B=160\,mT$', fontsize=14)


ax1 = fig.add_subplot(1,2,2)
c = get_color(0,10,'tab10')
ax1.errorbar(
    vert_result['Select'],
    [vert_result['Gamma'][T] for T in vert_result['Select']],
    [vert_result['Gamma_error'][T] for T in vert_result['Select']],
    fmt='o', color=c, label='Franz\' quasielastic')

ax1.set_ylabel(r'$\Gamma\ (\mu eV)$')
ax1.set_xlabel(r'$T\ (K)$')
ax1.set_xlim(27,31)
ax1.legend()

ax2 = ax1.twinx()
c = get_color(1, 10, 'tab10')
ax2.errorbar(
    elastic_result_1['Axis']['100_1'],
    elastic_result_1['Intensity']['100_1'],
    yerr=elastic_result_1['Intensity_error']['100_1'],
    fmt='o', color=c, label='Franz\' elastic')


#load Jonas' data
#Jdata = pd.read_csv('data/Kindervater_Bvertn_quasielastic', skiprows=6, names=['T1', 'Gamma'])
#c = get_color(0, 10, 'tab10')
#ax1.errorbar(Jdata['T1']+0.2, Jdata['Gamma'], fmt='s', mec=c, mew=2, label='Jonas\' elastic', color='None')

#Jdata = pd.read_csv('data/Kindervater_Bvertn_intensity', skiprows=6, names=['T2', 'Int'])
#c = get_color(1, 10, 'tab10')
#ax2.errorbar(Jdata['T2']+0.2, Jdata['Int']/(22.*3), fmt='s', mec=c, mew=2, label='Jonas\' quasielastic', color='None')
plt.savefig('B_vert_n.pdf')

# Plot parallel data

In [ ]:
############################################
#see what hapens
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.constants as co
from matplotlib.colors import Colormap, LogNorm
from matplotlib.cm import get_cmap
import pandas as pd
%matplotlib inline
fig = plt.figure(figsize=(16,5))
ax = fig.add_subplot(1,2,1)

Ts = []
Gammas = []
Gammaerrs = []

index = 0

def get_color(n, number_of_colors, cmapstr):

    cmap = get_cmap(cmapstr)

    return cmap(float(n)/number_of_colors)

print(parallel_result['Select'])
for T in parallel_result['Select']:
    
    x           = parallel_result['Parameters'][T]['x']
    y           = parallel_result['Parameters'][T]['y']
    print(y)
    y_error     = parallel_result['Parameters'][T]['y_error']

    c = get_color(index,10,'tab10')
    ax.errorbar(
        x, 
        y+1.2-index*0.2,
        y_error, 
        fmt='o', 
        color=c,
        label='$T=%.2f\,K$' %T)
    x = np.linspace(0.01,3,1000)

    ax.plot(x, parallel_result['Curve'][T]+1.2-index*0.2, color=c)
    
    index += 1


ax.set_xscale('log')
ax.set_xlim(0.01,3)
ax.legend(bbox_to_anchor=(-0.1,0.9))
ax.text(2e-2, 0, r'$B \perp n$, $B=160\,mT$', fontsize=14)


ax1 = fig.add_subplot(1,2,2)
c = get_color(0,10,'tab10')
ax1.errorbar(
    parallel_result['Select'],
    [parallel_result['Gamma'][T] for T in parallel_result['Select']],
    [parallel_result['Gamma_error'][T] for T in parallel_result['Select']],
    fmt='o', color=c, label='Franz\' quasielastic')

ax1.set_ylabel(r'$\Gamma\ (\mu eV)$')
ax1.set_xlabel(r'$T\ (K)$')
ax1.set_xlim(27,31)
ax1.legend()

ax2 = ax1.twinx()
c = get_color(1, 10, 'tab10')
ax2.errorbar(
    elastic_result_2['Axis']['110_1'], 
    elastic_result_2['Intensity']['110_1'], 
    yerr=elastic_result_2['Intensity_error']['110_1'], 
    fmt='o', color=c, label='Franz\' elastic')

#load Jonas' data
#Jdata = pd.read_csv('data/Kindervater_Bparan_quasielastic', skiprows=6, names=['T1', 'Gamma'])
#c = get_color(0, 10, 'tab10')
#ax1.errorbar(Jdata['T1']+0.4, Jdata['Gamma'], fmt='s', mec=c, mew=2, label='Jonas\' elastic', color='None')

#Jdata = pd.read_csv('data/Kindervater_Bparan_intensity', skiprows=6, names=['T2', 'Int'])
#c = get_color(1, 10, 'tab10')
#ax2.errorbar(Jdata['T2']+0.4, Jdata['Int']/(44*3), fmt='s', mec=c, mew=2, label='Jonas\' quasielastic', color='None')
plt.savefig('B_parallel_n.pdf')